------------------------------------------------------
*Applications of Machine Learning*
------------------------------------------------------

*Vanessa Gómez Verdejo vanessa@tsc.uc3m.es*




# Recommender Systems

# 1. Introduction

Recommender systems are one of the most common and easily understandable applications within machine learning and data science.

Based on information about users and contents (or items) and the preferences or tastes that users have for different contents, recommender systems make predictions about the interest that a user may have in new products that he may not have shown interest  in  (or bought) yet. This is called **personalised recommendations**.

Due to the large amount of information that surrounds us, many companies have chosen to include these systems within their services to cope with this information and provide the user with personalised recommendations, content, and/or services.

Some examples of these systems, with which we already naturally coexist, can be found in the following:


1. **AMAZON**

The online shopping giant incorporates a personalised recommendation algorithm on its website. When a user is interested in a product, the system provides two lists: the first related to the product search and the second with a set of products that may also be of interest to them based on their purchase history...

<img align="center" src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/Amazon.png" width=60%>


The aim of this system is to increase sales. In fact, Amazon has stated that 30% of its average sales are generated by recommendations.

What is Amazon's basis for generating this personalised list of recommendations? And how does it manage to do it efficiently with the large number of users and products it has to handle?

The idea is quite simple: for each product, Amazon generates a list of similar products that may be of interest along with this product. To do this, it analyzes the products that users have purchased in the same order, or added to the shopping cart, or simply stored in a their wish list. 

Amazon does not look at the characteristics of the products to see if one product is similar to another, but analyzes the users' preferences for the products to generate these similarities. In other words, if a user buys an iPad and jointly buys a case, when a new user buys the iPad, Amazon will recommend buying the case. This is what is known as recommendation systems based on **collaborative filtering**.


2. **SPOTIFY**

Spotify not only provides a platform to access all the music we want to listen to with great ease and speed, but since 2015 it also offers a weekly playlist with personalised recommendations. 

<img align="centre" src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/Spotify.png" width=60%>

In this case, the aim is not to increase sales, but to build user loyalty by preventing users from getting tired of always listening to the same thing. In fact, Spotify states that 70% of its plays come from these recommendations.

To design this recommendation system, Spotify combines 3 recommendation models:
1. Collaborative Filtering Models: they analyse users' tastes to recommend similar songs.
2. Natural Language Processing Models: analyse the information available on the Internet about the groups and/or songs to detect trends.
3. Audio models: analyse raw audio tracks to look for similarities between musical genres to help improve recommendations.

The combination of these 3 models gives rise to the Spotify recommender, which has earned the loyalty of many customers.


3. **NETFLIX**.

Netflix also includes a personalised recommendation system to help users find series and films that interest them. 

<img align="centre" src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/netflix.jpg" width=60%>

To design this system Netflix uses the information about users and content:
* View history together to the ratings over the already viewed  content. 
* Tastes and preferences of other users of the system.
* Information about the content: titles, genre, categories, actors, year of release, etc.
* User behaviour: time of day watched, devices, time spent using the system.

All this data feeds the recommendation system so that, combining schemes based on collaborative filtering and others based on content and user information, it can give personalised recommendations to users.

## Goals of this session

* Learn the main techniques for the design of recommender systems. We will pay special attention to collaborative filtering systems.
* Become familiar with the (python) libraries that can help us to design them. In particular, we will use the Surprise library that implements the most common collaborative filtering models.
* Learn how to design these models, properly validate their parameters, and evaluate their performance.
* Analyse the advantages and disadvantages of each of these methods.

To do this, in the following sections, we will go through a practical study case and we will design different recommendation systems.




# 2. Starting information: the database

For the design of recommender systems, we  usually have the following available information:

* Metadata or **descriptive** information **of the contents**.
* Sociodemographic **information of the users**.
* **Ratings** or user evaluation of the content. These ratings can be explicit or implicit:
  - Explicit information: when the user has rated a content by giving a specific score (e.g. a value on a scale from 1 to 5).
  - Implicit information: in some applications it is very difficult to ask the user to vote on content; in these cases, the user's browsing or system usage history can be followed to find out the user interests (pages visited, content previewed, number of times a song is listened to, etc.).

The main difficulty in designing these systems is that most of this information is not available. In fact, the main objective of these systems is to predict the ratings that users would give to content that they have not rated yet.



## **Book-Crossing dataset**
For this session we will work with a book recommendation database: [Book-Crossing](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)

This database contains information from 278,858 users (anonymised but with demographic information) who provide 1,149,780 ratings of some 271,379 books. This information is structured in 3 tables (or .csv files):

* **User table**: It contains the users' information: an identifier and if available some demographic data such as location and age. As this information has been anonymised, the user identifiers are integers.  
* **Book table**: For each book, its identifier (in this case the ISBN) and additional metadata such as book title, author, year of publication and publisher are provided.
* **Rating table**: It contains the information with the ratings that users have given to some books. The ratings are explicit and implicit. The explicit ratings are expressed on a scale from 1 to 10 (higher values denoting greater interest), and the implicit ratings are indicated with a value of 0.

Let's load the database and analyse the content in each of these matrices.

### Rating matrix

The score table is usually a matrix like this


<img align="centre" src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/Ratings.png" width=40%>

where most of the elements are not available. In other words, it is a very sparse matrix (usually only 1%-5% of the data is known). For this reason, this matrix is usually stored in a `(user_id, item_id, rating)` format. To see what this matrix looks like, let's start by loading the rating table and analysing how its values are distributed.

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None # To silect warning in chained_assignment

# Load the data file
url_ratings = "http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/BX-Book-Ratings.csv"

rating = pd.read_csv(url_ratings, sep=';', error_bad_lines=False, encoding="latin-1")
rating.columns = ['userID', 'bookID', 'bookRating']
rating.drop_duplicates(inplace=True)
# Check the data table and get some information
print('Number of entries in the rating table:',rating.shape[0])
print('Number of users:', len(rating.userID.unique()))
print('Number of books (items):', len(rating.bookID.unique()))
print('Maximum rating value:', rating.bookRating.max())
print('Minimum rating value:', rating.bookRating.min())

In [ ]:
rating.head()

The structure of this table is a tuple (`userID`, `bookID`, `rating`). The table collects one entry for each rating issued. Users who have not rated any book (e.g. new users in the system) have no entry in this table.

Rating distribution: What score values are there? Let's draw its histogram.

In [ ]:
import matplotlib.pyplot as plt

rating_distribution = rating.bookRating
plt.hist(rating_distribution)
plt.ylabel('Number of ratings')
plt.xlabel('Rating')
plt.show()

We can see that the majority of the ratings (700K out of 1200K scores in total) are 0 (implicit scores), and very few scores are 1, 2 or 3. In general, if a book achieves low scores it means that it has not been liked.

How many books has each user rated? To answer this question, let's draw the distribution of scores per user.


In [ ]:
# Count the number of entries in the rating matrix by user
Nrating_por_user = rating.userID.value_counts()
plt.hist(Nrating_por_user, range(20))
plt.ylim([1, 60000])
plt.xlim([1, 20])
plt.ylabel('Number of users')
plt.xlabel('Number of ratings')
plt.show()


print("The user who has rated most books has rated:", Nrating_por_user.max())

We see that this distribution has many values at the beginning and then shows a long tail... this shows that many users have rated very few books and there are few users with many rated books. Although there is one user who has rated 13602 books!!!!! 

And how many scores do we have for each book? Let's now show the distribution of ratings per book.



In [ ]:
Nrating_por_book = rating.bookID.value_counts()
plt.hist(Nrating_por_book, range(20))
plt.ylim([1, 200000])
plt.xlim([1, 20])
plt.ylabel('Number of books')
plt.xlabel('Number of ratings')
plt.show()

print("The highest rated book has:", Nrating_por_book.max())

We now see a similar distribution where most of the books received less than 5 ratings, and very few books have many ratings, although the highest rated book has received 2.502 ratings.

### Table of contents (books)

Let's now analyse the information in this table.

In [ ]:
url_books = "http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/BX-Books.csv"

books = pd.read_csv(url_books, sep=';', error_bad_lines=False, encoding="latin-1", usecols=["ISBN", "Book-Title", "Book-Author", "Year-Of-Publication"], low_memory=False)
books.columns = ['bookID', 'title', 'author', 'year']  
books.head()

In this case each row of the table is associated to a book and has the information of the book (ID, title, author and year).

### User table

Let's now analyse the information in the user table.

In [ ]:
url_users = "http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/BX-Users.csv"

users = pd.read_csv(url_users, sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'localication', 'age']  
users.head()

As in the content table, each row of the table is associated to a user and has the user's information. In this case their ID, location and year. We see that some fields are not available for some users and are listed as `NaN`.

### Data preprocessing

Before starting to work with this database, we are going to filter some of its fields based on two criteria:

1. We are going to eliminate implicit scores. Most of the models that we are going to see only work with explicit scores, so we are going to eliminate the implicit scores to facilitate the design of the models.

2. To reduce the size of the dataset, and avoid running into long runs, we are going to filter out books with few ratings and users who have scored few books. In fact, these users/books were not going to give us good results, so by eliminating them we facilitate the analysis of the methods we are going to study.




In [ ]:
print('The size of the original rating matrix is:\t{}'.format(rating.shape))
print('The size of the original user matrix is:\t{}'.format(users.shape))
print('The size of the original content matrix is:\t{}'.format(books.shape))

# 1. Remove implicit ratings

rating = rating[rating.bookRating>0]

# 2. Remove users and books with a low number of ratings

min_book_ratings = 50
filter_books = rating.bookID.value_counts() > min_book_ratings
filter_books = filter_books[filter_books].index.tolist()

min_user_ratings = 50
filter_users = rating.userID.value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

rating = rating[(rating.bookID.isin(filter_books)) & (rating.userID.isin(filter_users))]
print('The size of the filtered rating matrix is:\t{}'.format(rating.shape))

users = users[(users.userID.isin(filter_users))]
print('The size of the filtered user matrix is:\t{}'.format(users.shape))

books = books[(books.bookID.isin(filter_books))]
print('The size of the filtered content matrix is:\t{}'.format(books.shape))

# 3. Recommender systems: what are they and how are they classified?

As we have seen, recommender systems start from information about the contents (`books`), the users (`users`) and the users' interests in some of the contents (`rating`) in order to predict the users' interest in the unrated articles. In other words, fill in the blanks in the rating table.

Depending on the used information and how this information is used, we can classify recommender systems as:

*  **Popularity-based models**: these systems only consider the content popularity  (based on user ratings, number of sales,...). They are easy to implement and enjoy a certain level of effectiveness, but they do not provide personalised recommendations (they do not take into account the personal preferences of each user).

*  **Content-based systems**: the user is recommended similar items to those he liked in the past: "Show me more of the same items that I liked". This similarity between contents is done on the basis of the descriptions and characteristics of the contents (information about the books: similar title, same author, ...).

* **User-based systems**: a user is recommended based on the tastes of users with a similar socio-demographic profile (this approach is not usually used).

* **Collaborative filtering**: the user is recommended with items that were liked (in the past) by people with similar tastes and preferences. In other words, items rated positively by users who tend to agree with the user are recommended to the user.

* **Hybrids**: combine the above schemes to exploit the advantages and overcome limitations of the different approaches.

Two of the most popular approaches to recommender systems are collaborative filtering and content-based recommendations. In this notebook we are going to focus on collaborative filtering systems (the most common); but first let's review a couple of simple approaches to design a popularity-based system and a content-based system.

## 3.1 Popularity-based systems

As mentioned above, one of the simplest recommendation methods are popularity-based systems. This approach recommends the most important items based on a certain metric or score (popularity). 

In this section, we are going to design one of these systems for our book recommendation database. To do so, we will follow the steps below:
1. Select a popularity criterion, metric or score to rate the books.
2. Calculate the popularity of each book.
3. Sort the books according to their popularity to generate a list of recommendations.


One of the simplest metrics could be to consider the average rating to each book or directly select the books with a high number of scores.

Let's analyze these two metrics. To do so, complete the following exercises:

**Exercise 1**: Calculate the average rating for each book and show the 10 books with the highest average rating.

**Exercise 2**: Calculate the number of ratings for each book and display the 10 books with the highest number of ratings.

If you find it useful, you can use the `groupby` and `agg` functions of the `dataframes` (you can find their help [here](https://pandas.pydata.org/pandas-docs/version/0.23.1/generated/pandas.core.groupby.DataFrameGroupBy.agg.html) and [here](https://pandas.pydata.org/pandas-docs/version/0.23.1/api.html#id39)).


#### Solution

In [ ]:
# Exercise 1
#<SOL>
#</SOL>

In [ ]:
# Exercise 2
#<SOL>
#</SOL>

**Designing our popularity metric**.

What are the potential drawbacks of the above metrics? 

* Using only the number of ratings we see that it doesn't make much sense because the most rated books can have bad ratings.

* But using the average rating can give us problems because some books can have a very high score and be unreliable because they have few ratings... For example, "The Curious Incident of the Dog in the Night-T..." has a 9.3 with only 10 votes, and "Harry Potter and the Goblet of Fire" has an average score of 9.19 with 63 votes. Which would you recommend first with a highest confidence? 	

    
Looking at this, it seems most appropriate to use a metric that takes into account both the average rating and the accumulated number of votes. This system will ensure that, for example, a book with an average score of 6 with 500 votes will get a (much) higher score than a film with the same score but only a few votes.

IMDB ([Internet Movie Database](https://www.imdb.com/)) assigns the popularity of its films with this formula:
$$ {\rm Weighted~Rating~(WR)}=\frac{v}{v+m}R+\frac{m}{v+m}C$$
where,
   * $v$ is the number of ratings;
   * $m$ is the minimum number of votes required for its average rating to be significant;
   * $R$ average rating;
   * $C$ average rating over all items.

If $v>>m$, ${\rm WR} = R$, having many ratings we rely on its average rating; but if $m>>v$, ${\rm WR} = C$, we do not rely on the ratings and the assigned popularity is the average of the whole database.

Let's calculate this metric for our book collection.



**Exercise 3**: Calculate the WR index for each of the books of our database and show the 10 books with the highest index. Set the value of $m$ to 10.

#### Solution


In [ ]:
# m fixed to 10
m = 10

#<SOL>
#</SOL>


In [ ]:
import seaborn as sns
# To plot the result
plt.figure(figsize=(16,6))
sns.barplot(x='WR',y='title', data=book_rating.head(20))

plt.title('Libros más populares', weight='bold')
plt.xlabel('Índice WR', weight='bold')
plt.ylabel('Título', weight='bold')
plt.show()

In [ ]:
# To print the result
book_rating.head(20)

## 3.2 Content-based systems

In this section we are going to review content based systems. In this case, given a user, we are going to recommend books similar to the books he has already liked. To obtain the book similarities, we will use the descriptive information of each book. If we look at the `books` table, we see that we have the following information:


In [ ]:
books.head()

* **Title**: we could analyse the semantic content of the titles to find books with similar topics. In fact, for this type of processing it would be interesting to have a summary of the book that is more descriptive of its content and, then, you can use many of the approaches learnt in the NLP block (topic models, word2vec, ...).
* **Author**: in this case we can directly say that two books are similar if they were written by the same author.
* **Year**: we could consider the year as another similarity variable, although it would only make sense if we combine it with any of the previous criteria.

With this information and to build a simple system (not including NLP methods), we can apply the following criterion to build a list of similar books: "*two books are similar if they are by the same author and are more similar if they have been published closer in time*".

To build this system let's solve the following exercises:


### **Exercise 4**
Complete the following code cell to given a `bookID`, build a dataframe (`books_author`) with all the books of the `bookID` author.

*Note*: you can use the `.loc()` method of the DataFrames to make selection of rows that meet a certain condition.

#### Solution

In [ ]:
bookId = '0345402871'	

# Preprocess authors: convert all characters to lowercase
books['author'] = books['author'].str.lower()

#<SOL>
#</SOL>


###**Exercise 5**
For the `books_author` dataframe include a new column with the difference in years between the `bookId` publication year and the rest of the books in the dataframe. Then, build the `similar_books` dataframe by sorting the rows of `books_author` so that the books closest in time to `bookID` come first. It may be advisable to remove the reference book entry (the row with `bookID='0345402871'`) from this dataframe.

#### Solution

In [ ]:
# Preprocess years: convert to integer
books_author['year'] = books_author['year'].astype(int)

#<SOL>
#</SOL>

**Exercise 6**
Let's now combine the solutions of Exercises 4 and 5 to build a function that, given a `bookId`, returns the list of most similar books (by the same author and closest in time).

#### Solution

In [ ]:
def find_similar_books(bookId):
  #<SOL>
  #</SOL>
  return similar_books

bookId = '0345402871'	

similar_books = find_similar_books(bookId)

similar_books

### And how do we make recommendations to a user?

Now that we know how to find similar books, we just need to look at the books that the user has previously rated a positive (he liked them) in order to recommend the books similiar to these.

In [ ]:
rating.head()

In [ ]:
# We choose a user and see their previous likes and dislikes.

userId = 277427

ratings_userID = rating.loc[rating['userID'] == userId]
ratings_userID.sort_values(ascending=False,by=['bookRating']) 

We see that this user really liked the books: `002542730X`, `0345413903`, ..., `0743412028` so we can recommend books similar to these...

In [ ]:
list_books = ['002542730X', '0345413903', '0061009059', '0385486804', '0399501487', '0553280368', '0679731148', '0743412028']
for bookId in  list_books:
  title = books.loc[books['bookID'] == bookId]['title']

  print('Libros parecidos a '+title+':' )
  similar_books = find_similar_books(bookId)
  print(similar_books['title'].values)

# Collaborative filtering

Collaborative filtering systems look for similarities between users/contents based on the ratings that users have given to the contents. They do not make use of user demographic information or metadata associated with the contents.

These systems are divided into two types:

1. **User-based filtering**: these systems make recomendations to a user based in the contents liked by users with similar preferences. For example, let's say Alice and Mary have similar tastes in books (i.e. they largely like and dislike the same books). Let's say a new book has been released and Alice has read it and liked it. Therefore, it is very likely that Mary would also like it, and therefore, the system would recommend this book to Mary.

2. **Content-based filtering**: these systems are similar to the system we have just designed (content-based), except for the fact that the similarities between the contents (or items) are calculated based on the ratings given to them by the users. For example, if Alice, Mary and Anne have given 5 stars to the Harry Potter and Twilight books, the system identifies the items as similar. Therefore, if someone buys one of the Harry Potter books, the system also recommends the Twilight saga.

To design these systems we only need the rating matrix!!!

## 4.1 User-based collaborative filtering

Given the "active user" (Alice) and an item ($i$) that she has not yet seen, these systems work as follows:
1. They find the set of users who are most similar to Alice (they liked similar objects) and, in addition, have rated the item $i$.
2. Then, to predict the rating that Alice would give  to the item $i$, they use the average of the ratings that these users (those who have similar preferences to Alice) have given to item $i$.

Applying this process on all the objects that Alice has not rated yet, we could obtain some predictions of their scores with which we can make a ranking and, then, recommend to Alice the ones that we believe would have a higher rating.

Let's see in detail how to carry out this process with the following example:

<img align="center" src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/CF_user2user_1.png" width=60% >


**Step 1**: Let's calculate the similarity between users. 

To measure whether two users $u$ and $v$ are similar, it is common to use the correlation coefficient between the vectors of common scores, i.e., given the subset ($I_{u,v}$) of items that both users have scored, their similarity is given by:
$${\rm sim}(u,v) = \frac{\sum_{i \in I_{u,v}}  r_{u,i}  r_{v,i}  }{\sqrt{\sum_{i \in I_{u,v}} r_{u,i}^2  \sum_{i \in I_{u,v}}r_{v,i}^2}}$$
where $r_{u,i}$ is the score that user $u$ has given to item $i$. Thus, we can have that Alice has the following similarities with the rest of the users:

<img align="centre" src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/CF_user2user_2.png" width=80%>




**Step 2**: Let's make predictions about the ratings.

Now to find out the rating that Alice would give to Item 5 we can take the most similar users to Alice ($N_A$) and average the scores that these users have given to Item 5, weighting them by their similarities, i.e.:
$$ \hat{r}_{A,Item5} = \frac{\sum_{v \in N_A} {\rm sim}(A,v) ~r_{v,Item5} }{\sum_{v \in N_A} {\rm sim}(A,v)} $$

In our example, we only use users 1 and 2 as similar users, so we would have:
$$ \hat{r}_{A,Item5} = \frac{0.85-3 + 0.7-5 }{0.85+ 0.7} = 3.9 $$ 

In general, for a user $u$ the estimated score on item $i$ is given by
$$ \hat{r}_{u,i} = \frac{\sum_{v \in N_u^K} {\rm sim}(u,v) ~r_{v,i} }{\sum_{v \in N_u^K} {\rm sim}(u,v)} $$
where $N_u^K$ are the $K$ most similar neighbours to the user $u$.




Although in this example users are rating items on a scale between 0 and 5, there is no guarantee that all users will interpret the scale equally or that some users will be more or less generous in their ratings. To correct for these effects, we may find modifications to the above model by including normalised versions of the ratings.

Normally, there are two variants:
* **Average correction**: in this case, the average value of the user's scores is taken away from each score, i.e, 
$$ r_{u,i}^{norm} = r_{u,i} - \bar{r_{u}}$$
where $\bar{r_{u}}$ is the mean value of the scores given by the user $u$.
* **z-score correction**: in this case both the mean and the standard deviation are adjusted, resulting in a new rating or score called z-score that tries to make the distribution of the scores of all users the same. This new normalised score is given by:
$$ r_{u,i}^{norm} = \frac{r_{u,i} - \bar{r_{u}}}{std(r_{u})}$$

where $\bar{r_{u}}$ and $std(r_{u})$ are, respectively, the mean value and the standard deviation of the ratings given by the user $u$.


## 4.2 Content-based collaborative filtering

The working principles of this system are very similar to the previous one, but we will use the similarities between objects (and not between users) to make the predictions.
For example, going back to our example

<img align="centre" src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/CF_item2item.png" width=60%>

To do this, we can measure the similarity between items $i$ and $j$ as the correlation between the ratings given by the users who have rated both items
$${\rm sim}(i,j) = \frac{\sum_{u \in U_{i,j}}  r_{u,i}  r_{u,j}  }{\sqrt{\sum_{u \in U_{i,j}} r_{u,i}^2  \sum_{u \in U_{i,j}}r_{u,j}^2}}$$

where $U_{i,j}$ is the set of users who have rated both items. Applying this expression to our example, we see that the items most similar to Item 5 are Item 1 and Item 4. 

We can then use the ratings given by Alice to the set of contents similar to Item 5 ($N_{Item5}$) to make predictions:


$$ \hat{r}_{A,Item5} = \frac{\sum_{j \in N_{Item5}} {\rm sim}(Item5,j) r_{A,j} }{\sum_{j \in N_{Item5}} {\rm sim}(Item5,j)} $$

which for our example would give us that $\hat{r}_{A,Item5}=4.7$.

For a general case, the estimated rating for a user $u$ on item $i$ is given by
$$ \hat{r}_{u,i} = \frac{\sum_{j \in N_i^K} {\rm sim}(i,j) r_{u,j} }{\sum_{j \in N_i^K} {\rm sim}(i,j)}$$

where now $N_i^K$ are the $K$ most similar neighbours to item $i$.

Finally, if we wish, we can also apply the mean or z-score corrections to homogenise the ranges of scores between users.

And now that I know some of the collaborative filtering models, how do I apply them to my book recommendation problem?

We could implement these models in Python and design our own functions, but here we are going to make use of one of the most widely used Python libraries for Recommender Systems: [Surprise](http://surpriselib.com/)

# 5. SURPRISE

The [SurPRISE](http://surpriselib.com/) library (Simple Python RecommendatIon System Engine) facilitates the design of recommendation systems (especially collaborative filtering systems) by including tools for:

* Manage data (especially rating matrices). Users can use built-in datasets (Movielens, Jester) or own databases.
* Train multiple collaborative filtering algorithms: the ones we have just seen based on neighbourhood between users or between content, as well as other collaborative filtering methods that we will see later.
* Facilitate the evaluation, analysis and comparison of the performance of different methods. 
* Perform parameter tuning. This includes cross-validation procedures.

All these methods or tools have a very similar syntax to those of sklearn, so they are very easy to use.

Let's see how to work with this library on our example. Let's start by installing Surprise in Colab.

In [ ]:
!pip install scikit-surprise

## 5.1. Loading and preprocessing data in Surprise

To load a dataset from a dataframe into *Surprise*, we can use the `load_from_df()` method. To do this, we need a dataframe with three columns (in this order): 
* user ids,
* content ids,
* assigned scores/ratings.

In addition, this function uses a `Reader` object where we have to specify with the `rating_scale` parameter the range of the rating values (in our case from 1 to 10).




In [ ]:
from surprise import Reader, Dataset

reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(rating[['userID', 'bookID', 'bookRating']], reader)

Let's see the before and after...

In [ ]:
rating.head()

In [ ]:
print(type(data))
data.raw_ratings[:10]

*Note*: The last column is used in datasets that have predefined partitions, in our case there are none and their values are set to `None`.

To train our model, as when working with classification/regression models, it is appropriate to split the training set into training and test partitions. To do this we can use the `train_test_split()` function.

In [ ]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

Looking at this code, everything looks very simple and similar to our working procedure in sklearn. But **be careful**, Surprise handles different data types, and when we apply these conversions it generates different data types and it uses internal indexes for users and items...

In [ ]:
print(type(trainset))
# This object allows to obtain the tuples (id_u, id_i, rating) with the .all_ratings() method.
print('Rating values')
print(list(trainset.all_ratings())[:10])
# We can use the functions .to_raw_iid(), .to_raw_uid to convert the internal ids to the original ones.
# Or the functions .to_inner_iid() and .to_inner_uid() to undo this conversion.

In [ ]:
print(type(testset))
testset[:10]

As we will see, these objects, `trainset` and `testset`, are the ones used, respectively, by the `.fit()` and `.predict()` methods of the different models. If we want to train a model with the original dataset, we would have to convert the `surprise.dataset.DatasetAutoFolds` data type to `surprise.trainset.Trainset` or we will have to turn it into a `list` if we want to make predictions. This can be done with the functions `build_full_trainset()` and `build_testset()` or `build_anti_testset()` as shown in the following example:

In [ ]:
# To check we print the number of ratings in data
print('Original data')
print(len(data.raw_ratings))

# We generate a set of ratings with the training data.
trainset_all = data.build_full_trainset()
print('Training data')
print(type(trainset_all))
print(trainset_all.n_ratings)

# We generate a test set with the training data (to compute training  performance).
testset_all = trainset_all.build_testset()
print('Test data (with the taining samples)')
print(type(testset_all))
print(len(testset_all))

# We generate a test set with the data that are not in training. 
# (all combinations of unrated users and items) 
testset_all_anti = trainset_all.build_anti_testset()
print('Test data (with data that are not in training)')
print(type(testset_all_anti))
print(len(testset_all_anti))


## 5.2 Training a model

As Sklearn, to train a model we just need to define the model with its constructor and then use the `.fit()` method to train it. To train a model based on neighbours, Surprise includes the following methods:

* [`KNNBasic`](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNBasic): this is the basic collaborative filtering algorithm we have seen before. It implements collaborative filtering based on users or content as defined by the `user_based` parameter of [`sim_options`](https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measures-configuration).

* [`KNNWithMeans`](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithMeans): the same algorithm as above but with mean correction.

* [`KNNNWithZScore`](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithZScore): the same algorithm as the previous ones but with mean and standard deviation correction.  

When defining these models, Surprise allows us to select two parameters that affect the neighbourhood used to make the predictions. If we remember that the predictions are given by:

$$ \hat{r}_{u,i} = \frac{\sum_{v \in N_u^K} {\rm sim}(u,v) r_{v,i} }{\sum_{v \in N_u^K} {\rm sim}(u,v)} $$

where $N_u^K$ are the $K$ most similar neighbours to the user $u$, Surprise lets us control this neighbourhood with these parameters:
* `k`: number of neighbours (users/items) that are considered to make the prediction (this is the size of the set $N_u^K$). If a user has more neighbours, only the `k` most similar are used. And if it has fewer, only those are used, unless they are less than `k_min`.
* `k_min`: minimum number of required neighbours of a user/item to make a prediction. For users/items with less than `k_min` neighbours, their predictions are maded based on their average ratings, i.e. on $\bar{r}_{u}$ or  $\bar{r}_{i}$.

Moreover, the class [`sim_options`](https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measures-configuration) not only allows us to select a model based on either users or contents, but also allows us to define several metrics, imported from the [`similarities`](https://surprise.readthedocs.io/en/stable/similarities.html#module-surprise.similarities) module, to find the most similar users or items to a given one. Specifically, it includes 4 metrics:

* `cosine`: Calculates the cosine similarity between all pairs of users (or items). For example, between users $u$ and $v$ this measure would be:
$${\rm sim}(u,v) = \frac{\sum_{i \in I_{u,v}}  r_{u,i}  r_{v,i}  }{\sqrt{\sum_{i \in I_{u,v}} r_{u,i}^2  \sum_{i \in I_{u,v}}r_{v,i}^2}}$$
where $I_{u,v}$ is the set of items that both users have scored.

* `msd`: Calculates the similarity as the inverse of the root mean square distance between all pairs of users (or items). In this case the similarity between users $u$ and $v$ is given by:
$${\rm sim}(u,v) = \frac{1}{1 + \sum_{i \in I_{u,v}}  \left(r_{u,i} - r_{v,i} \right)^2}$$
where a $+1$ term is included in the denominator to avoid divisions by $0$.

* `pearson`: Calculates the Pearson correlation coefficient between all pairs of users (or items), given by:
$${\rm sim}(u,v) = \frac{\sum_{i \in I_{u,v}}  \left(r_{u,i} - \bar{r_{u}}\right)  \left(r_{v,i}- \bar{r_{v}} \right) }{\sqrt{\sum_{i \in I_{u,v}} \left(r_{u,i} - \bar{r_{u}}\right)^2  \sum_{i \in I_{u,v}} \left( r_{v,i} - \bar{r_{v}}\right)^2}}$$
where $\bar{r_{u}}$  and $\bar{r_{v}}$ are the mean values of the user ratings $u$ and $v$.

* `pearson_baseline`: This is the same measure as before, except that instead of subtracting the averages from the ratings, it subtracts some mean coefficients that must be learnt (we will see later how to learn them). In addition, it includes a correction (*shrinkage*) to avoid overfitting effects when there are few common elements between users $u$ and $v$. With these changes this measure is given by:
$${\rm sim}(u,v) =\frac{\mid I_{u,v} \mid -1}{\mid I_{u,v} \mid -1+Sh} \frac{\sum_{i \in I_{u,v}}  \left(r_{u,i} - {b_{u}}\right)  \left(r_{v,i}- {b_{v}} \right) }{\sqrt{\sum_{i \in I_{u,v}} \left(r_{u,i} - {b_{u}}\right)^2  \sum_{i \in I_{u,v}} \left( r_{v,i} - {b_{v}}\right)^2}}$$
where ${b_{u}}$ and ${b_{v}}$ are the  mean coefficients of users $u$ and $v$ (which we will have to learn) and $Sh$ is the *shrinkage* coefficient which is a model parameter to be defined.

In addition, the similarity calculation includes the parameter `min_support` which indicates the minimum number of common items for the similarity to be calculated. If two users (or two items) do not have a minimum of `min_support` common elements scored, their similarity will be 0.

Now that we know the models included in Surprise, let's see how to define and train one of these models. Let's start with the simplest model, the `KNNBasic` with its default parameters.

In [ ]:
from surprise.prediction_algorithms.knns import KNNBasic

KNNalgo = KNNBasic()
KNNalgo.fit(trainset)

# Check the model parameters
print(KNNalgo.k)
print(KNNalgo.min_k)
print(KNNalgo.sim_options)

If we want, we can modify these parameters in the constructor. For example, we can set a neighbourhood of 60 and a `k_min` value of 10.

In [ ]:
KNNalgo = KNNBasic(k=60, min_k=10)
KNNalgo.fit(trainset)

# Check the model parameters
print(KNNalgo.k)
print(KNNalgo.min_k)
print(KNNalgo.sim_options)

We can even change the `sim_options` setting to make a content-based model using the `msd` as a measure of similarity between items.

In [ ]:
sim_options = {'name': 'msd', 'user_based': False}
KNNalgo = KNNBasic(k=60, min_k=10, sim_options = sim_options )
KNNalgo.fit(trainset)

# Check the model parameters
print(KNNalgo.k)
print(KNNalgo.min_k)
print(KNNalgo.sim_options)

## 5.3 Getting predictions 

Once the model is trained, in Surprise we can use the method [`.predict()`](https://surprise.readthedocs.io/en/stable/predictions_module.html#surprise.prediction_algorithms.predictions.Prediction) to get the prediction of a particular item for a given user. Or if we want to get the predictions for a test set (several users and items at once) we can use the `.test()` method. 


If we activate the `verbose` of the `predict` method, it returns information about the prediction. This information includes the actual value (if we tell it to do so when we call it), the predicted value and a `PredictionImpossible` flag that tells us if the prediction was possible. For example, in KNN methods if there are not enough neighbours (`min_k`) the prediction cannot be made and this method returns this exception and as  prediction provides the average value of the ratings.


#### **Exercise 7**: 
Estimate the rating of the first 4 items in the test set.

#### Solution

In [ ]:
# Check the four first elements in the test dataset
testset[:4]

In [ ]:
#<SOL>
#</SOL>

In [ ]:
#<SOL>
#</SOL>

#### **Exercise 8**:
Estimate the ratings of the whole test set.

#### Solution


In [ ]:
#<SOL>
#</SOL>

## 5.4. Evaluating the model

Once we have the model predictions, we can use some of the following measures or metrics in the [accuracy](https://surprise.readthedocs.io/en/stable/accuracy.html) module to evaluate the model performance. In particular there are 4 measures to evaluate the error:

* `rmse`: Calculates the root mean squared error (RMSE, *Root Mean Squared Error*):
$$ RMSE = \sqrt{{1}{N} \sum_{i=1}^N (p_i -r_i)^2}$$
* `mse`: Calculates the mean squared error (MSE, *Mean Squared Error*):
$$ MSE = \frac{1}{N} \sum_{i=1}^N (p_i -r_i)^2$$
* `mae`: Calculates the mean absolute error (MAE, *Mean Absolute Error*):
$$MAE = \frac{1}{N} \sum_{i=1}^N |p_i -r_i|$$
* `fcp`: Calculates the fraction of concordant pairs (FCP, *Fraction of Concordant Pairs*). This measure is oriented to evaluate a ranking result and not the accuracy of the estimated vs the actual rating values. To do so, they measure the number of concordant pairs per user by counting the number of pairs:
$$n^u_c=\mid \{(i,j), ~ \hat{r}_{u,i}>\hat{r}_{u,j} ~~\rm{y} ~~~{r}_{u,i}>{r}_{u,j}\}\mid$$
and equivalently evaluate the number of discordant pairs $n^u_d$. Then, averaging these values over all users, $n_c = \sum_u n^u_c$ and $n_d = \sum_u n^u_d$, the FCP is defined as:
$$ FCP = \frac{n_c}{n_c+n_d}$$

To use these metrics we only need to call the corresponding function within the `accuracy` module by giving it the set of predictions we have estimated (note that in these predictions the actual rating values are also included, so the error can be calculated).


In [ ]:
from surprise import accuracy

# Compute performance 
RMSE = accuracy.rmse(predictions)

MSE = accuracy.mse(predictions)

MAE = accuracy.mae(predictions)

FCP = accuracy.fcp(predictions)

## 5.5 Analysis of the influence of model parameters

So far, we have trained a simple KNN model, based on similarities between users, and we have used its default parameters.

In this section, we will see how the values of `K` and `k_min` influence on the model performance, the differences using a model based on user or content similarity, as well as the impact of using different similarity measures. Finally, we will see whether using other versions of the KNN model, such as those that include mean or z-score corrections, can provide advantages.

To do so, please complete the following exercises.

#### **Exercise 9**:
Analyse the influence of the value of `k` in the `KNNBasic` model (keep the remaining  parameters to their default values). Explore values in the range $[10,20,30,40,50]$ and measure the model performance in terms of RMSE.

#### Solution

In [ ]:
#<SOL>
#</SOL>

#### **Exercise 10**:
Analyse the influence of the value of `min_k` on the `KNNBasic` model (set `k` to the best value from the previous section and keep the reamining parameters to their default values). Explore values in the range 1 to 20 and measure performance in terms of RMSE.

#### Solution

In [ ]:
#<SOL>
#</SOL>

#### **Exercise 11**:
Analyse the differences between using a user-based or content-based model and, in addition, the influence of using different similarity measures (`msd`, `cosine`, `pearson`, `pearson_baseline`). You can set `k` and `min_k` to the best values from the previous sections. Again, measure performance in terms of RMSE.

#### Solution

In [ ]:
#<SOL>
#</SOL>

#### **Exercise 12**:
Finally, analyse whether the modified versions of the `KNNBasic` model (`KNNWithMeans` and `KNNWithZScore`) provide any advantages over the basic model. You can use for this analysis the best parameter settings obtained in the previous sections.

#### Solution

In [ ]:
from surprise.prediction_algorithms import KNNBasic, KNNWithMeans, KNNWithZScore

#<SOL>
#</SOL>

## 5.6 Parameter validation

Surprise provides several tools to run cross-validation procedures to find the best parameters for a given algorithm. The tools are easy to use, as they follow the sklearn format.

On the one hand, it has functions to generate training and test partitions (like the one we have used above) or to generate different validation partitions within the training set (in this [link](https://surprise.readthedocs.io/en/stable/model_selection.html#module-surprise.model_selection.split) you can find a detail of all of them).

On the other hand, similar to sklearn  `GridSearch` module, Surprise includes the `GridSearchCV` function that allows us to define a model, the parameters to explore and it does all the magic for us.

To see how it works, let's train a basic neighbourhood recommendation model (`KNNBasic`) based on content (`'user_based': False`) and select by CV the number of neighbours (`k`) and the minimum number of neighbours (`min_k`). 

Note: The Gridsearch function is designed to work with data of type `surprise.dataset.DatasetAutoFolds`. So, for this example, we will forget about the training/test partitions we have created and we will validate the model using all the data in `data`. Later we will see how we can use our training and test partitions.

In [ ]:
from surprise.model_selection.search import GridSearchCV

param_grid = {'k': range(10,50,10), 'min_k': range(1,10),
              'sim_options': {'name': ['cosine'],
                              'user_based': [False]}}
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse', 'mse'], cv=3)

gs.fit(data)

We can analyse the results of the CV process by analysing the content of `cv_results`.

In [ ]:
results_df = pd.DataFrame.from_dict(gs.cv_results)
results_df.head()

Or we can directly see the best result and optimal parameters in `best_score` and `best_params`.

In [ ]:
# Mejor RMSE 
print(gs.best_score['rmse'])

# Parametros óptimos
print(gs.best_params['rmse'])

If we want to apply a CV process using train/test partitions as we have done so far, we cannot use the `trainset` and `testset` objects with `GridsearchCV`. So we have to create our own dataset objects with train and test partitions. The following example shows how to do this.

In [ ]:
import random      
import copy                                                        

# We start with the complete dataset in dataset
raw_ratings = data.raw_ratings                                             

# We randomise the ratings (to generate random partitions)                                
random.shuffle(raw_ratings)                                                

# 75% trainset, 25% testset                                                
threshold = int(.75 * len(raw_ratings))                                     
trainset_raw_ratings = raw_ratings[:threshold]                             
test_raw_ratings = raw_ratings[threshold:]                                 

datatrain = copy.deepcopy(data)
datatrain.raw_ratings = trainset_raw_ratings  

#datatest = copy.deepcopy(data)
#datatest.raw_ratings = test_raw_ratings  
                                                     
# We apply CV with our new Dataset that has only the train partition.   
print('GRID SEARCH...')         

param_grid = {'k': range(10,50,10), 'min_k': range(1,10),
              'sim_options': {'name': ['msd'],
                              'user_based': [False]}}
# We set refit=True to re-train the final model with the whole dataset.                         
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=3, refit= True)
gs.fit(datatrain)

In [ ]:
# Ahora evaluamos en el conjunto de test                                            
testset_gs = data.construct_testset(test_raw_ratings)      
predictions = gs.test(testset_gs)
print('RMSE en la particion de test:')                                          
RMSE = accuracy.rmse(predictions)                  

## 5.7 Limitations of neighbourhood-based models

* How can we make recommendations to new users of the system or to new content? This is known as the **cold-start-problem** and the solution is to create hybrid systems that combine collaborative filtering systems with systems based on popularity, user or content. For example:
  * The problem of new users can be alleviated with a hybrid system that incorporates user demographic information to find similar users. Or asking new users for rating a minimum number of objects before the system can start working.
  * The problem of new content can also be alleviated by hybrid systems that, in this case, incorporate metadata information of the contents. Or by asking users for rating new items (and giving them some reward for doing so).

* And even if users and content are not new, sometimes the ratings they have are too few to make good estimates and ratings are directly estimated by averages. Surprise includes a threshold on the minimum number of common scores between two users or items that are arequired to calculate similarities. 

* Computational cost of these models. As we have seen how these models work, to estimate the score of a user $u$ for an item $i$, we need to calculate the users who are similar to user $u$ (in a user-based model) or contents that are similar to item $i$ (in the content-based model). This computation is computationally expensive as it requires calculating the distances between all users or items (by accessing all scores of all users or items) in order to make an estimate; these models are known as **memory-based** and can only be used (from a practical point of view) if the database is not very large. To overcome this limitation, it is sometimes proposed to pre-compute these distances (knowing which users or items are similar to a given one), generating a **model-based** system, and then only averaging scores to predict new ratings. This scheme tends to work very well in content-based neighbourhood models since the similarities between items do not tend to vary over time; in fact, this is the scheme used by Amazon in its recommendation system and it is able to make recommendations on millions of products in a very efficient way.



## 6. Collaborative Filtering Models based on latent or factorial decompositions

## 6.1. Introduction 

In latent factor models users and items are mapped to a lower dimensional space. Their new vector representation, in this new space, is called latent. Let's see this process with the example from before:

<img align="center" src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/Factorizacion1.pdf" width=80%>

This latent representation has several advantages:
1. By having the latent representation of the users and items, we can reconstruct (or estimate) the matrix of scores. 

<img align="center" src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/Factorizacion2.pdf" width=80%>

2. This new representation of users and items can help to interpret the predictions:

<img align="centre" src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/ML/SR/latent-factors.png" width=50%>

(*) Figure from https://kevinkolcheck.com/wp-content/uploads/2017/12/latent-factors.png



And how can we obtain this latent representation? With the **Alternating Least Squares** (ALS) algorithm. 

The ALS algorithm tries to approximate the rating matrix by factoring it as the product of two matrices: 

$$ R = X * Y $$

where $X$ and $Y$ are the latent factor matrices describing the properties of each user and each item, respectively.

These matrices are computed to minimise the mean squared error between the predictions and the actual ratings over the known ratings (those of our training set):

$$ \min_{X, Y} \sum_{(u, i) \in P} \left( r_{u,i} - \bf{x}_u^{\top} \bf{y}_i \right)^2 + \lambda \left( ||X||^2 + ||Y||^2 \right)
$$

where P is the set of elements with known rating.

To find the solution to this problem, the ALS algorithm starts by randomly initialising the latent factor matrices $X$ and $Y$ and then, considering the values of $Y$ are known, optimises the values of $X$ such that :

$$ \min_{X} \sum_{(u, i) \in P} \left( r_{u,i} - \bf{x}_u^{\top} \bf{y}_i \right)^2 + \lambda ||X||^2 $$

Then, with the latent factors obtained for the users ($X$), the latent values of the items are optimized:
$$ \min_{Y} \sum_{(u, i) \in P} \left( r_{u,i} - \bf{x}_u^{\top} \bf{y}_i \right)^2 + \lambda |||Y|||^2 $$

Iterating over these two steps until the algorithm converges, we reach the final lantent factor for users and items.

The fact that this algorithm alternates between a problem in $X$ and a problem in $Y$ is what makes this method called *Alternating Least Squares*.

This method is a collaborative filtering system with model, since it performs a previous training where it calculates the latents and to make predictions it only needs to do the product of the associated latents (it does not need to re-access the whole rating matrix).

## 6.2. Latent models in Surprise

Surprise includes the implementation of this model within the [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html) module where it provides a more generalised implementation; instead of considering that a score is estimated by the product of its associated latents ($\hat{r}_{u,i} = \bf{x}_u^{\top} \bf{y}_i$) the predictions are considered to be given by this model:
$$\hat{r}_{u,i} = \mu +  b_u + b_i + \bf{x}_u^{\top} \bf{y}_i$$

where $\mu$ is the mean of the scores, ${b_{u}}$ and ${b_{i}}$ are the coefficients of means of user $u$ and item $i$, and $\bf{x}_u$ and $\bf{y}_i$ are the latent variables of user $u$ and item $i$. To learn all these parameters, Surprise minimises the mean squared error (as the generic ALS does), but using this new definition of  $\hat{r}_{u,i}$ and learning all the parameters together. 

However, the model includes a `biased` parameter that we can set to `False` if we want to remove all these biasing factors and recover the original ALS model. In addition, this model includes as input parameter the number of latents (number of components in the vectors $\bf{x}_u$ and $\bf{y}_i$), which we will have to specify through the parameter `n_factors` and as we will see will be critical to obtain good performance with this model.

Finally, Surprise includes the SVD method and has two variants of it ([SVDpp](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVDpp) and [NMF](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.NMF)) that allow, respectively, to work with implicit scores and to impose different restrictions on the latent factors. If you are interested and want to go deeper into these models, you can check the library documentation.

## 6.2.1 Training and evaluation of the model

We start by training the SVD model with all its default parameters and evaluating its performance. To do this, we are going to reuse the training and test partitions we had.

In [ ]:
from surprise.prediction_algorithms.matrix_factorization import SVD

algoLatent = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algoLatent.fit(trainset)
predictions = algoLatent.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

As we can see, without adjusting any parameter, this model is already giving performances below 1.58 (best result so far). Let's now see how this method behaves using or not the biasing factors and how the number of latents influences its performance. To do so, complete the following exercises.

#### **Exercise 13**:
Analyse the performance of the SVD algorithm, in terms of RMSE, in its basic version (without biasing factors, `biased = False`) for different number of latents. You can explore values of this parameter in the range from 1 to 100 with steps of 5.

#### Solution

In [ ]:
# We evaluate the evolution of the RMSE with the number of latents and without mean corrections.
range_nlatents =  range(1,100, 5)
RMSE_nlatents = []
for nlatents in range_nlatents:
  print('Value of nlatents: ' + str(nlatents))
  #<SOL>
  #</SOL>
plt.figure()
plt.plot(range_nlatents, RMSE_nlatents)


#### **Exercise 14**:
Repeat the previous exercise using the mean factors (`biased = True`) and analyse the results.

In [ ]:
# We evaluate the evolution of the RMSE with the number of latents and with mean corrections.

range_nlatents =  range(1,100, 5)
RMSE_nlatents = []
for nlatents in range_nlatents:
  print('Value of nlatents: ' + str(nlatents))
  #<SOL>
  #</SOL>
plt.figure()
plt.plot(range_nlatents, RMSE_nlatents)

**Conclusions**: As we can see, it seems that the full model (with mean factors) tends to show better results and to be less dependent on the number of latents used. Whereas in the previous case, the choice of the number of latents is critical (in this database using few latents gives good results, but this is **not** the usual scenario). 

### 6.2.2 Parameter validation

Let's analyze the model performance  by selecting this parameter (`n_factors`) with cross-validation. We will use our approach to redefine the training and test partitions in order to compare with the results obtained so far.

In [ ]:
# Usamos las variables datatrain y testset_gs que definimos antes

param_grid = {'n_factors': range(1,100, 5), 'biased': [True]}

# Fijamos refit=true para que reentrene el modelo final con todo el conjunto de datos                         
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, refit= True)
gs.fit(datatrain)

# Ahora evaluamos en el conjunto de test                                            
testset_gs = data.construct_testset(test_raw_ratings)      
predictions = gs.test(testset_gs)
print('RMSE en la particion de test:')                                          
RMSE = accuracy.rmse(predictions)    

### 6.2.3 Latent analysis

Let's now analyse the latents obtained. To do so, we will train a model with only two latents and we will represent items in this (two-dimensional) latent space.

We can carry out a similar analysis with the users, but we do not have information about them to be able to extract any conclusion.

In [ ]:
algoLatent = SVD(n_factors = 2, biased=False, random_state =20)

# Train the algorithm on the trainset, and predict ratings for the testset
algoLatent.fit(trainset)

X = algoLatent.pu
Y = algoLatent.qi

In [ ]:
import numpy as np
items_id_Y = np.array([trainset.to_raw_iid(id) for id in trainset.all_items()])
latentsY = np.concatenate((items_id_Y[:, np.newaxis], Y), axis=1)

pd_items_id_Y = pd.DataFrame(latentsY, columns = ['bookID', 'Y0', 'Y1'])
pd_items_id_Y.Y0 = pd_items_id_Y.Y0.astype(float)
pd_items_id_Y.Y1 = pd_items_id_Y.Y1.astype(float)
pd_items_id_Y = pd.merge(pd_items_id_Y, books[['bookID','title', 'year']], on='bookID', how='inner')
pd_items_id_Y.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))


pd_items_id_Y = pd_items_id_Y.loc[pd_items_id_Y.year>1998]
# Point size related to the year
years = pd_items_id_Y.year.values
scale = 2*(years - 1994)
ax.scatter(pd_items_id_Y.Y0.values, pd_items_id_Y.Y1.values, c='blue', s=scale, label='Items')

ax.legend()
ax.grid(True)

titles = pd_items_id_Y.title.values
#for i, txt in enumerate(titles):
for i, txt in enumerate(titles[:100]):
    ax.annotate(txt, (pd_items_id_Y.Y0.values[i], pd_items_id_Y.Y1.values[i]), label='Title')


#ax.scatter(X[:, 0], X[:, 1], s=3, c='red')

plt.show()


# Other models included in Surprise

**Random score generator**.

It considers that the scores follow a Gaussian distribution with mean $\mu$ and variance $v$ and when it is asked for a prediction, it generates a random value from that distribution.

The values of $\mu$ and $v$ are calculated from the sample means and variances of the scores in the training set.

[NormalPredictor](https://surprise.readthedocs.io/en/stable/basic_algorithms.html#surprise.prediction_algorithms.random_pred.NormalPredictor)

**Mean-based generator**

This model assumes a user rating for an item is given by:

$$r_{u,i} = \mu + b_u + b_i$$

As you can see, this model is a simplified version of the SVD and the learning of these parameters is also done using the ALS.

[BaselineOnly](https://surprise.readthedocs.io/en/stable/basic_algorithms.html#surprise.prediction_algorithms.baseline_only.BaselineOnly)

**K-NN with baseline**

Modifies the neighbourhood-based model by including mean estimates. That is, using the above model carries out a first estimation of the ratings:
$$b_{u,i} = \mu + b_u + b_i$$

And then these estimatimations are used as a mean correction to estimate the ratings with a neighbourhood approximation

$$ \hat{r}_{u,i} = b_{u,i} + \frac{\sum_{v \in N_u} {\rm sim}(u,v) \left( r_{v,i}- b_{v,i}\right) }{\sum_{v \in N_u} {\rm sim}(u,v)} $$

[KNNBaseline](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNBaseline)

**Slope One**
In this case, the predictions are given by:

$$ \hat{r}_{u,i} = \bar{r}_{u} + \sum_{j \in R_{u,i}}\frac{dev(i,j)}{\mid U_{i,j} \mid} $$

where $R_{u,i}$ is the set of items rated by user $u$ and where the deviation between the scores of item $i$ and item $j$, $dev(i,j)$, are measured as

$$dev(i,j) = \sum_{u \in U_{i,j}}\frac{r_{u,i}-r_{u,j}}{\mid U_{i,j} \mid}$$

where $U_{i,j}$ is the set of users who have rated items $i$ and $j$, and ${\mid U_{i,j} \mid}$  is the number of items in this set.

[SlopeOne](https://surprise.readthedocs.io/en/stable/slope_one.html#surprise.prediction_algorithms.slope_one.SlopeOne)

**Co-clustering**

This algorithm carries out a clustering of users and items, so you have $G$ groups of users and $H$ groups of items. For each of these groups it calculates:
* $C_{g}$: average rating of users in the $g$-th group.
* $C_{h}$: average rating of the items in the $h$-th group.
* $C_{g,h}$: average rating of the ratings of the users in the $g$-th group and the items in the $h$-th group.

If we further precompute the average score of each user and of each item, which we denote as $\bar{r_{u}}$ and $\bar{r_{i}}$, the predicion of the score of user $u$ (which belongs to group $g$) for item $i$ (which belongs to group $h$) is given by:
$$ \hat{r}_{u,i} = C_{g,h} + \left(\bar{r_{u}} -  C_{g}\right) + \left(\bar{r_{i}} -  C_{h}\right)$$



# Embeddings for colaborative filtering: Prod2Vec

We have just seen how the ALS method is able to obtain a low-dimensional representation of users and products. From this representation, we can obtain estimates of the scores that users would give to items that they have not yet scored. However, this low-dimensional representation could be used for other tasks, since by having each item/user characterised with a vector we can find similarities between them, do clustering, classification, .....

Another way to obtain a vector representation (embedding) of the products of our recommender system is to borrow same ideas and tools from word2vec representation to obtain embeddings of products instead of words, for that reason this approach is called Prod2Vec.

Let's see how to obtain this Prod2Vec for a shopping recommendation system$^{(*)}$. To do so, let's start by loading the dataset.

$^{(*)}$ This dataset is a simplified version of the challenge "Instacart market basket analysis" from [Kaggle](https://www.kaggle.com/c/instacart-market-basket-analysis).

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None # To silect warning in chained_assignment
product_df = pd.read_csv("http://www.tsc.uc3m.es/~vanessa/data_notebooks/market_basket/products.csv")
order_product_df = pd.read_csv("http://www.tsc.uc3m.es/~vanessa/data_notebooks/market_basket/market_basket_red_v2.zip").set_index('Unnamed: 0')


In this dataset we have two data tables, the first indicates the product information (it indicates what each `product_id` is).

In [ ]:
product_df

And on the other hand, the table with orders, for each purchase, indicates the products that have been bought and, in addition,  the order in which they are purchased. That is to say, it indicates the sequence of purchase of products.

In [ ]:
order_product_df.head()

### Construction of the Prod2Vec representation

We can exploit the sequential structure given by the `order_product_df` table to see which products are usually accompanied by others, i.e., to define the context of each product. To do this, we only need to create sentence-like structures with the order in which products are purchased within the same order.

In [ ]:
order_product_list = order_product_df[['order_id','product_id']].values.tolist()

product_corpus = []
sentence = []
new_order_id = order_product_list[0][0]
for (order_id, product_id) in order_product_list:
    if new_order_id != order_id:
        product_corpus.append(sentence)
        sentence = []
        new_order_id = order_id
    sentence.append(str(product_id))


Let's look at the content of these lists

In [ ]:
# Function to help to analyze products
def toProductName(id):
    id = int(id)
    return product_df[product_df.product_id==id]['product_name'].values.tolist()[0]
toProductName(24852)

In [ ]:
# First sentence or first shopping list
id_list = 0
list_products = [toProductName(id_prod) for id_prod in product_corpus[id_list]]
print(list_products)

In [ ]:
# Second sentence or first shopping list
id_list = 1
list_products = [toProductName(id_prod) for id_prod in product_corpus[id_list]]
print(list_products)

Now that we have our "corpus" of products or shopping lists, we can use the gensim `word2vec` function to build a `prod2vec`.

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(product_corpus, window=6, size=100, min_count=200)


### What are the most similar products?

We can use the prod2vec representation or the product embedding to find similar products:

* The most similar product to `banana` (24852) is ...

In [ ]:
def most_similar_readable(model, product_id):
    similar_list = [(product_id,1.0)]+model.wv.most_similar(str(product_id))
    
    return [( toProductName(int(id)), similarity ) for (id,similarity) in similar_list]

In [ ]:
pd.DataFrame(most_similar_readable(model, 24852), columns=['product','similarity'])

* The closest product to `Organic Whole Milk` (27845) is ... 

In [ ]:
pd.DataFrame(most_similar_readable(model, 27845), columns=['product','similarity'])

## Recommending products with prod2vec

Now, we can use these similarities to design a content based colaborative filtering system.

For this purpose, let's follow these steps:

### Build the rating matrix

In [ ]:
df_ratings = order_product_df.groupby(['user_id','product_id']).size().reset_index(name='ratings')
df_ratings 

### Make predictions

As we already know how to compute similarities, we can directly obtain predictions.

For a general case, the estimated rating for a user $u$ on item $i$ is given by
$$ \hat{r}_{u,i} = \frac{\sum_{j \in N_i^K} {\rm sim}(i,j) r_{u,j} }{\sum_{j \in N_i^K} {\rm sim}(i,j)}$$

where now $N_i^K$ are the $K$ most similar neighbours to item $i$.


In [ ]:
import numpy as np

# Obtain the contents included into prod2vec
list_products =list(model.wv.vocab)

# We choose a user (we can iterate it over all users)
user_id = 206208 	
df_user = df_ratings.loc[df_ratings['user_id']==user_id]

# We choose an item (we can iterate this over all products)
for product_id in ['26384']:# list_products:
  # Compute similar products to product_id and save this information                      
  sim_products = model.wv.most_similar(product_id, topn=40)
  df_user_sim_aux = df_user
  df_user_sim_aux['sim']=0
  for sim in sim_products:
    if int(sim[0]) in df_user_sim_aux['product_id'].values:
      df_user_sim_aux['sim'].loc[df_user_sim_aux['product_id']==int(sim[0])] = sim[1]
 
  # We can print the similar products
  print('List of similar products')
  print(df_user_sim_aux.loc[df_user_sim_aux['sim']>0])
  # Now, we can make the prediction
  pred_rating = (df_user_sim_aux['ratings']*df_user_sim_aux['sim']).sum()/df_user_sim_aux['sim'].sum()
  print('The predicted scoring is: %2.2f' %pred_rating)

## Other  Applications of prod2vec: product grouping

We can use the prod2vec representation to do product grouping by using it as input to a K-means. To do this we will train a k-means with 500 groups.

In [ ]:
from sklearn.cluster import KMeans
K=500

# Get product embeddings
prod_emb = model.wv.vectors

# Train Kmeans
kmeans = KMeans(n_clusters=K) # Definimos objeto con parámetros por defecto
kmeans.fit(prod_emb) # Entrenamos k-means
y_kmeans = kmeans.predict(prod_emb) # Obtenemos el identificador del grupo para cada dato
centers = kmeans.cluster_centers_

Let's analyse the resulting clusters by looking at the products most similar to each centroid.

In [ ]:
def most_similar_to_centroid(model, center_emb):
    similar_list = model.wv.similar_by_vector(center_emb)
    
    return [( toProductName(int(id)), similarity ) for (id,similarity) in similar_list]

In [ ]:
center_id = 0
pd.DataFrame(most_similar_to_centroid(model, centers[center_id]), columns=['product','similarity'])

In [ ]:
center_id = 100
pd.DataFrame(most_similar_to_centroid(model, centers[center_id]), columns=['product','similarity'])

In [ ]:
center_id = 200
pd.DataFrame(most_similar_to_centroid(model, centers[center_id]), columns=['product','similarity'])